# Fire Radiative Energy to Total Particulate Matter Workbook

This workbook develops the workflow for generating estiamtes of total particulate matter (TPM) from observations of fire radiative energy (FRE).  The FRE observations are derived from time integrated observations of fire radiative power (FRP), which is a measure of the amount of energy emitted by a fire per unit time and has units $Wm^{-2}$.  In order integrate over time we need observations of the FRP through time, and therefore rely on observations of fires from geostrationary sensors, in particular: SEVIRI over Africa/Europe; GOES over North and South America; Himawari over Asia and Oceania.  Which provide observations at 15, 30 and 10 minute intervals respectively.  

The TPM ($gm^{-2}$) observations are required to be instantaneous only, as such they can be derived from polar orbiting sensors.   The TPM estimates are derived from optical depth (OD, unitless) estimates taken from the smoke plumes associated with a fire as the ratio OD to the mass extinction coffecient ($m^{2}g^{-1}$).  As such for TPM we need two parameters.  OD is a commonly retrived parameters from remotely sensed optical imagery, for example MODIS has a number of products which come under the MOD04/MYD04 product classification.  The mass extinction coefficient is typically derived from in-situ observations. 

The basic premise is to develop FRE observations for a set of fires, specifcally selecting fires with clear and definable smoke plumes, calculating the TPM for these plumes and finally deriving a linear relation between the FRE and the TPM.  Once this relation is defined it is then possible to determine the TPM directly from observations of FRE.

Here we explore the development of this relation.

In [19]:
# load in required packages
import ast
import glob
import os
import re
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
from netCDF4 import Dataset
from pyhdf.SD import SD, SDC
from matplotlib.path import Path
from scipy import stats
from scipy import integrate
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from mpl_toolkits.basemap import Basemap

import src.config.filepaths as filepaths
import src.config.sensor as sensor

import matplotlib.pyplot as plt
import scipy.ndimage as ndimage

In [2]:
root_path = '/Users/dnf/Projects/kcl-fire-aot/data/'

Load in the FRP for the himawair sensor (as currently working on SE Asia).  Keep only the columns we need, convert lat lon columns to a shapely Point class, and set the index to datetime.  

In [3]:
def load_frp():
    if sensor.sensor == 'himawari':
        try:
            frp_df = pd.read_pickle(filepaths.path_to_himawari_frp + 'himawari_df.p')
        except Exception, e:
            print('could not load frp dataframe, failed with error ' + str(e) + ' building anew')
            frp_files = os.listdir(filepaths.path_to_himawari_frp)
            df_from_each_file = (pd.read_csv(os.path.join(filepaths.path_to_himawari_frp, f)) for f in frp_files)
            frp_df = pd.concat(df_from_each_file, ignore_index=True)

            # lets dump the columns we don't want
            frp_df = frp_df[['FIRE_CONFIDENCE', 'FRP_0', 'LATITUDE', 'LONGITUDE', 'year','month','day','time']]

            # make geocoords into shapely points
            points = [Point(p[0], p[1]) for p in zip(frp_df['LONGITUDE'].values, frp_df['LATITUDE'].values)]
            frp_df['point'] = points
            frp_df.drop(['LONGITUDE', 'LATITUDE'], axis=1, inplace=True)

            # reindex onto date
            for k in ['year', 'month', 'day', 'time']:
                frp_df[k] = frp_df[k].astype(int).astype(str)
                if k == 'time':
                    frp_df[k] = frp_df[k].str.zfill(4)
                if k in ['month', 'day']:
                    frp_df[k] = frp_df[k].str.zfill(2)

            format = '%Y%m%d%H%M'
            frp_df['obs_time'] = pd.to_datetime(frp_df['year'] +
                                             frp_df['month'] +
                                             frp_df['day'] + 
                                             frp_df['time'], format=format)
            frp_df['obs_date'] = frp_df['obs_time'].dt.date
            frp_df.drop(['year','month','day', 'time'], axis=1, inplace=True)
            
            frp_df.to_pickle(filepaths.path_to_himawari_frp + 'himawari_df.p')

    elif sensor.sensor == 'goes':
        pass

    return frp_df
frp_data = load_frp()

In [11]:
def build_polygon(plume, orac_data):

    # TODO replace this with ORAC data, not using L1B data
    # get geographic coordinates of plume bounds (first test with l2 data)
    myd_data = SD(os.path.join(filepaths.path_to_modis_l1b, plume.filename), SDC.READ)
    lats = ndimage.zoom(myd_data.select('Latitude').get(), 5)
    lons = ndimage.zoom(myd_data.select('Longitude').get(), 5)
    
    # when digitising points are appended (x,y).  However, arrays are accessed
    # in numpy as row, col which is y, x.  So we need to switch
    bounding_lats = [lats[point[1], point[0]] for point in plume.plume_extent]
    bounding_lons = [lons[point[1], point[0]] for point in plume.plume_extent]

    return Polygon(zip(bounding_lons, bounding_lats))

Load in the smoke plume mask:

In [8]:
mask_path = root_path + 'Asia/processed/plume_masks/myd021km_plumes_df.pickle'
try:
    mask_df = pd.read_pickle(mask_path)
except:
    mask_df = pd.read_csv(mask_path, quotechar='"', sep=',', converters={'plume_extent': ast.literal_eval})

In [6]:
def compute_fre(plume, frp_data):
        
    # subset temporally, getting day of overpass and also day before
    plume_date_stop = datetime.strptime(plume.filename[10:17], '%Y%j').date()
    plume_date_start = plume_date_stop - timedelta(days=1)
    try:
        frp_subset = frp_data.loc[(frp_data['obs_date'] == plume_date_stop) | 
                                   (frp_data['obs_date'] == plume_date_start)]
    except Exception, e:
        print 'Could not extract time subset, failed with error:', str(e)
        return None
        
    # subset spatially finding only those fires within the bounds of the plume
    # note Matplotlib path might be a better option to check with bounds
    # see here: https://goo.gl/Cevi1u
    inbounds = []
    plume_polygon = build_polygon(plume, [])
    try:
        for i, (index, frp_pixel) in enumerate(frp_subset.iterrows()):
            if frp_pixel['point'].within(plume_polygon):
                inbounds.append(i)
        if inbounds:
            frp_subset = frp_subset.iloc[inbounds]
    except Exception, e:
        print 'Could not extract spatial subset, failed with error:', str(e)
        return None
           
    # group by time and then aggregate the FRP variables appropriately
    frp_subset['FIRE_CONFIDENCE_mean'] = frp_subset['FIRE_CONFIDENCE']
    frp_subset['FIRE_CONFIDENCE_std'] = frp_subset['FIRE_CONFIDENCE']
    frp_subset = frp_subset.groupby('obs_time').agg({'FRP_0':np.sum, 
                                                     'FIRE_CONFIDENCE_mean':np.mean,
                                                    'FIRE_CONFIDENCE_std':np.std})[['FRP_0', 
                                                                                    'FIRE_CONFIDENCE_mean',
                                                                                    'FIRE_CONFIDENCE_std']]
    
    # get the plume observation overpass UTC time
    plume_obs_time = datetime.strptime(re.search('[0-9]{7,7}\.[0-9]{4,4}', plume.filename).group(0), '%Y%j.%H%M')
    plume_obs_time_less_12_hours = plume_obs_time - timedelta(hours=12)
    
    # we can then get all FRPs in the 12 hours before the overpass
    frp_subset = frp_subset[(frp_subset.index <= plume_obs_time) & 
                            (frp_subset.index >= plume_obs_time_less_12_hours)]
    
    # set up the times for integration
    try:
        t0 = frp_subset.index[0]
        sample_times = (frp_subset.index - t0).total_seconds()
    except Exception, e:
        print 'Could not extract spatial subset, failed with error:', str(e)
        return None
 
    # now integrate 
    fre = integrate.trapz(frp_subset['FRP_0'], sample_times)
    print 'fre:', fre
    return fre

In [24]:
def compute_plume_area(plume):
    
    # first build shapely polygon
    plume_polygon = build_polygon(plume, [])
    
    # TODO is sinusoidal proj good enough?  Need to switch to UTM
    # see comments in here: https://goo.gl/SD8DMQ
    m = Basemap(projection='sinu',lon_0=0,resolution='c')
    
    # apply to shapely polygon
    projected_plume_polygon = transform(m, plume_polygon)
    
    # compute projected polygon area in m2
    return projected_plume_polygon.area 

In [25]:
for index, plume in mask_df.iterrows():
    
    # get collocated FRP data
    #fre = compute_fre(plume, frp_data)
    
    # get area of the plume
    #plume_area = compute_plume_area(plume)
    
    # get plume AOD
    total_aod = compute_aod


429.236432962
146.218483304
182.055742545
291.462876117
57.9166283466
366.010844294
849.740686129
1628.12844786
48.1796810977
1451.60646121
216.432362451
82.0213554104
94.1781304772
34.3869077103
56.0873899275
205.801174762
91.1305491972
135.544714291
30.1802928873
57.7678622871
53.4177646326
447.004319979
38.1844545896
62.4305853531
32.5283997883
74.8114425084
209.938449174
70.0502085591
23.0581558319
22.9210575367
167.718341151
65.4773671701
34.1292779758
55.8681003397
1175.26625833
88.7010548588
34.2281073981


KeyboardInterrupt: 